In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/AN2DL/Homework1

# Import libraries

In [ ]:
import numpy as np
import os
import random
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
plt.style.use('ggplot')

from pathlib import Path

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Model

from pathlib import Path

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

# Global settings and seed

In [ ]:
IMAGE_SHAPE = [96,96]
INPUT_SHAPE = (*IMAGE_SHAPE,3)
BATCH_SIZE = 16
SEED = 42
DATASET_DIR = Path() / 'training_data_final'
EPOCHS = 30

tf.random.set_seed(SEED)

# Instantiate the dataset generators


Create two generators, one for training and one for validation.
It should be more correct to create one generator only and select `subset='training'` and `subset='validation'`, but in this way augmentation is applied to both sets. I prefer to create [two generators with the same split and the same seed.](https://stackoverflow.com/questions/71744605/keras-imagedatagenerator-validation-split-does-not-split-validation-data-as-expe)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train generator with augmentation
train_image_gen  = ImageDataGenerator(rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      zoom_range=[0.5,1.5],
                                      brightness_range=[0.5,1.5],
                                      shear_range=0.2,
                                      vertical_flip=True,
                                      horizontal_flip=True,
                                      fill_mode='reflect',
                                      validation_split = 0.15,
                                      )

# validation generator without augmentation
validation_image_gen = ImageDataGenerator(validation_split = 0.15)

train_dataset = train_image_gen.flow_from_directory(directory=DATASET_DIR,
                                                    target_size=IMAGE_SHAPE,
                                                    color_mode='rgb',
                                                    classes=None,
                                                    class_mode='categorical',
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=SEED,
                                                    subset='training',
                                                    )

validation_dataset = validation_image_gen.flow_from_directory(directory=DATASET_DIR,
                                                              target_size=IMAGE_SHAPE,
                                                              color_mode='rgb',
                                                              classes=None,
                                                              class_mode='categorical',
                                                              batch_size=BATCH_SIZE,
                                                              shuffle=False,
                                                              seed=SEED,
                                                              subset='validation'
                                                              )

In [ ]:
OUTPUT_DIR = Path() / 'supernet_histories'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Supernet: VGG16

In [ ]:
vgg16  = keras.applications.vgg16.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224,3)
)
vgg16.trainable = False

In [ ]:
inputs = keras.Input(shape=INPUT_SHAPE)

x = layers.Resizing(224, 224, interpolation='bicubic', name='resizing')(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = vgg16(x)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, kernel_initializer=keras.initializers.HeUniform(seed=SEED))(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(8, activation='softmax', kernel_initializer=keras.initializers.GlorotUniform(seed=SEED))(x)

model_vgg16 = keras.Model(inputs, outputs)

chosen_opt = keras.optimizers.Adam(learning_rate=1e-3)
model_vgg16.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=chosen_opt, metrics=['accuracy'])

In [ ]:
history_VGG16 = model_vgg16.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, verbose=0)

In [ ]:
np.save(str(OUTPUT_DIR / 'history_VGG16.npy'), history_VGG16.history)

# Supernet: Xception

In [ ]:
Xception  = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(299, 299,3)
)
Xception.trainable = False

In [ ]:
inputs = keras.Input(shape=INPUT_SHAPE)

x = layers.Resizing(299, 299, interpolation='bicubic', name='resizing')(inputs)
x = keras.applications.xception.preprocess_input(x)
x = Xception(x)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, kernel_initializer=keras.initializers.HeUniform(seed=SEED))(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(8, activation='softmax', kernel_initializer=keras.initializers.GlorotUniform(seed=SEED))(x)

model_Xception = keras.Model(inputs, outputs)

chosen_opt = keras.optimizers.Adam(learning_rate=1e-3)
model_Xception.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=chosen_opt, metrics=['accuracy'])

In [ ]:
history_Xception = model_Xception.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, verbose=0)

In [ ]:
np.save(str(OUTPUT_DIR / 'history_Xception.npy'), history_Xception.history)

# Supernet: EfficientNetB0

In [ ]:
EfficientNetB0  = keras.applications.EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224,3)
)
EfficientNetB0.trainable = False

In [ ]:
inputs = keras.Input(shape=INPUT_SHAPE)

x = layers.Resizing(224, 224, interpolation='bicubic', name='resizing')(inputs)
x = keras.applications.efficientnet.preprocess_input(x)
x = EfficientNetB0(x)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, kernel_initializer=keras.initializers.HeUniform(seed=SEED))(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(8, activation='softmax', kernel_initializer=keras.initializers.GlorotUniform(seed=SEED))(x)

model_EfficientNetB0 = keras.Model(inputs, outputs)

chosen_opt = keras.optimizers.Adam(learning_rate=1e-3)
model_EfficientNetB0.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=chosen_opt, metrics=['accuracy'])

In [ ]:
history_EfficientNetB0 = model_EfficientNetB0.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, verbose=0)

In [ ]:
np.save(str(OUTPUT_DIR / 'history_EfficientNetB0.npy'), history_EfficientNetB0.history)

# Supernet: EfficientNetB7

In [ ]:
EfficientNetB7  = keras.applications.EfficientNetB7(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224,3)
)
EfficientNetB7.trainable = False

In [ ]:
inputs = keras.Input(shape=INPUT_SHAPE)

x = layers.Resizing(224, 224, interpolation='bicubic', name='resizing')(inputs)
x = keras.applications.efficientnet.preprocess_input(x)
x = EfficientNetB7(x)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, kernel_initializer=keras.initializers.HeUniform(seed=SEED))(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(8, activation='softmax', kernel_initializer=keras.initializers.GlorotUniform(seed=SEED))(x)

model_EfficientNetB7 = keras.Model(inputs, outputs)

chosen_opt = keras.optimizers.Adam(learning_rate=1e-3)
model_EfficientNetB7.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=chosen_opt, metrics=['accuracy'])

In [ ]:
history_EfficientNetB7 = model_EfficientNetB7.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, verbose=0)

In [ ]:
np.save(str(OUTPUT_DIR / 'history_EfficientNetB7.npy'), history_EfficientNetB7.history)

# Supernet: ResNet-50

In [ ]:
ResNet50  = keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224,3)
)
ResNet50.trainable = False

In [ ]:
inputs = keras.Input(shape=INPUT_SHAPE)

x = layers.Resizing(224, 224, interpolation='bicubic', name='resizing')(inputs)
x = keras.applications.resnet.preprocess_input(x)
x = ResNet50(x)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, kernel_initializer=keras.initializers.HeUniform(seed=SEED))(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(8, activation='softmax', kernel_initializer=keras.initializers.GlorotUniform(seed=SEED))(x)

model_ResNet50 = keras.Model(inputs, outputs)

chosen_opt = keras.optimizers.Adam(learning_rate=1e-3)
model_ResNet50.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=chosen_opt, metrics=['accuracy'])

In [ ]:
history_ResNet50 = model_ResNet50.fit(train_dataset, epochs=EPOCHS, validation_data=validation_dataset, verbose=0)

In [ ]:
np.save(str(OUTPUT_DIR / 'history_ResNet-50.npy'), history_ResNet50.history)

# Final Plot 

Read the history.

In [ ]:
names = ['VGG16', 'Xception', 'EfficientNetB0', 'EfficientNetB7', 'ResNet-50']
history = list()
for i in names:
    name = OUTPUT_DIR / 'history_'+i+'.npy'
    history.append(np.load(name, allow_pickle='TRUE').item())

In [ ]:
# Two subplots, the axes array is 1-d
fig, (ax1,ax2) = plt.subplots(2, sharex=True)

#fig.suptitle('Results with different supernets')

for i in range(0,len(names)):
  ax1.plot(history[i]['val_accuracy'], label=names[i])

ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epochs')
ax1.legend(loc='upper left')

for i in range(0,len(names)):
  ax2.plot(history[i]['val_loss'], label=names[i])

ax2.set_ylabel('Loss')
ax2.set_xlabel('Epochs')
ax2.legend(loc='upper left')

plt.savefig(str(Path() / 'report_material' / 'supernet.pdf'), bbox_inches='tight')
plt.show()